In [1]:
import pandas as pd
import numpy as np
import sys
import re
import requests 
from bs4 import  BeautifulSoup
import json
from tqdm import tnrange, tqdm_notebook
from datetime import datetime
from tqdm import trange,tnrange, tqdm_notebook
import time
import polars as pl

In [2]:
import pyodbc
from requests_html import AsyncHTMLSession
asession = AsyncHTMLSession()
from datetime import date
from dateutil.relativedelta import relativedelta

In [3]:
import plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
from plotly.offline import iplot, plot
py.offline.init_notebook_mode(connected=True)
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [4]:
idx = pd.IndexSlice
pd.options.display.float_format = '{:,.3f}'.format

In [57]:
PL_Y.loc[idx[list_bank,:],['IsAudit','ISB27','ISB30','ISB41','ISA20']].reset_index().to_excel('PL_Y_bank.xlsx')

In [58]:
BS_Y.loc[idx[list_bank,:],['IsAudit','BSA78','BSA53','BSB103','BSB105']].reset_index().to_excel('BS_Y_bank.xlsx')

In [32]:
def set_ind(x):
    x = x.set_index(['Ticker','YearReport','LengthReport'],inplace=True)
# BS = BS.rename(columns=dict(zip(sql_col['item'],sql_col['col'])))
# PL = PL.rename(columns=dict(zip(sql_col['item'],sql_col['col'])))
# CF = CF.rename(columns=dict(zip(sql_col['item'],sql_col['col'])))    
BS = BS.loc[:,~BS.columns.duplicated()]
PL = PL.loc[:,~PL.columns.duplicated()]
CF = CF.loc[:,~CF.columns.duplicated()]
set_ind(BS)
set_ind(PL)
set_ind(CF)
BS = BS.sort_index(level=[0,1,2])
PL = PL.sort_index(level=[0,1,2])
CF = CF.sort_index(level=[0,1,2])

In [33]:
df = pd.concat([BS.iloc[:,13:],PL.iloc[:,13:],CF.iloc[:,14:]],axis=1)
df = df.sort_index(level=[0,1,2])

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

BalanceSheetId OrganCode ComTypeCode   
Ticker YearReport LengthReport                                         
       2022       5                    5931118      IVNB          NH  \
                  5                    6022203     SHBVN          NH   
ABB    2022       1                    5265072       ABB          NH   
                  2                    5446823       ABB          NH   
                  3                    5589514       ABB          NH   
...                                        ...       ...         ...   
VPB    2022       3                    5569320       VPB          NH   
                  4                    5732830       VPB          NH   
                  5                    5732817       VPB          NH   
                  6                    5447145       VPB          NH   
                  9                    5569316       VPB          NH   

                                LengthSeries   StartDate     EndDate   
Ticker YearReport LengthReport                                         
       2022       5                       12  2022-01-01  2022-12-31  \
                  5                       12  2022-01-01  2022-12-31   
ABB    2022       1                        3  2022-01-01  2022-03-31   
                  2                        3  2022-04-01  2022-06-30   
                  3                        3  2022-07-01  2022-09-30   
...                                      ...         ...         ...   
VPB    2022       3                        3  2022-07-01  2022-09-30   
                  4                        3  2022-10-01  2022-12-31   
                  5                       12  2022-01-01  2022-12-31   
                  6                        6  2022-01-01  2022-06-30   
                  9                        9  2022-01-01  2022-09-30   

                               SourceName  Note  PublicDate  IsAudit  ...   
Ticker YearReport LengthReport                                        ...   
       2022       5                  None  None  2023-03-15     True  ...  \
                  5                  None  None  2023-06-09     True  ...   
ABB    2022       1                  None  None  2022-04-21    False  ...   
                  2                  None  None  2022-08-23     True  ...   
                  3                  None  None  2022-10-21    False  ...   
...                                   ...   ...         ...      ...  ...   
VPB    2022       3                  None  None  2022-10-20    False  ...   
                  4                  None  None  2023-01-30    False  ...   
                  5                  None  None  2023-03-28     True  ...   
                  6                  None  None  2022-08-09     True  ...   
                  9                  None  None  2022-10-20    False  ...   

                                BSA278 BSI279 BSI280 BSI281  BSI282 BSI283   
Ticker YearReport LengthReport                                               
       2022       5               None   None   None   None    None   None  \
                  5               None   None   None   None    None   None   
ABB    2022       1               None   None   None   None    None   None   
                  2               None   None   None   None    None   None   
                  3               None   None   None   None    None   None   
...                                ...    ...    ...    ...     ...    ...   
VPB    2022       3               None   None   None   None    None   None   
                  4               None   None   None   None    None   None   
                  5               None   None   None   None    None   None   
                  6               None   None   None   None    None   None   
                  9               None   None   None   None    None   None   

                               BSI284 BSI285 BSI286 BSI287  
Ticker YearReport LengthReport                              
       2022       5

In [51]:
df.to_excel('bank.xlsx')

# CRAWL BÁO CÁO MBKE

In [15]:
import os
# dir path for saving file E:/Report/MBKE
dir_path = 'E:/Report/MBKE'
#scan for files in the directory
files = os.listdir(dir_path)
#extract the number in the beginning of the file name
files = [re.findall(r'^\d+',x) for x in files]
#flatten the list
files = [item for sublist in files for item in sublist]
#convert the list to integer
files = [int(x) for x in files]
#find the maximum number in the list
start = max(files)
end = start + 1000
file_list = os.listdir(dir_path)
list_file = pd.DataFrame(file_list, columns=['filename'])
list_file = list(list_file['filename'])


In [18]:
link = 'https://www.kelive.com/KimEng/servlet/PDFDownloadViaEmail?source=0&rid='
list_link = pd.DataFrame()
list_link['id_file'] = range(start,end)
list_link['link'] = link + list_link['id_file'].astype(str)
r = requests.get(list_link.loc[0,'link'],timeout=5)
for i in tnrange(len(list_link)):
    try:
        r = requests.get(list_link.loc[i,'link'],timeout=2)
        try:
            with r:
                fname = ''
                if "Content-Disposition" in r.headers.keys():
                    fname = re.findall("filename=(.+)", r.headers["Content-Disposition"])[0]
                else:
                    fname = list_link.loc[i,'link'].split("/")[-1]
            list_link.loc[i,'filename'] = fname[1:-1]
            if list_link.loc[i,'filename'] not in list_file:
                with open(dir_path+list_link.loc[i,'filename'], "wb") as f:
                    f.write(r.content)
            else: print('có rồi')

        except requests.exceptions.RequestException as e:
            print(e)
    except requests.exceptions.RequestException as e:
        list_link.loc[i,'status'] = 'ERROR'
        print("Error", list_link.loc[i,'link'])


C:\Users\HP PAVILION\AppData\Local\Temp\ipykernel_19568\2303806787.py:6: TqdmDeprecationWarning:

Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`



  0%|          | 0/1000 [00:00<?, ?it/s]

Error https://www.kelive.com/KimEng/servlet/PDFDownloadViaEmail?source=0&rid=60690
Error https://www.kelive.com/KimEng/servlet/PDFDownloadViaEmail?source=0&rid=60694
Error https://www.kelive.com/KimEng/servlet/PDFDownloadViaEmail?source=0&rid=60701
Error https://www.kelive.com/KimEng/servlet/PDFDownloadViaEmail?source=0&rid=60703


KeyboardInterrupt: 

# XỬ LÝ DF CÔNG TY LISTED

In [6]:
df = pd.read_pickle('../DataSQL/df_merge')

In [7]:
nganh = nganh.rename(columns={'ticker':'Ticker'})

In [20]:
df.loc[idx['VIC',2023:,[9]],['st_debt','lt_debt','te','cce','net_st_inv']]/10**9

,,,st_debt,lt_debt,te,cce,net_st_inv
Ticker,year,quarter,,,,,
VIC,2023,9.000,"95,183.876","98,292.858","146,942.516","16,882.015","7,369.738"


In [9]:
df = pd.merge(df.reset_index(),nganh[['Ticker','ICB2','ICB3','ICB4','ICB5']],on='Ticker')
df.set_index(['Ticker','year','quarter'],inplace=True)

In [10]:
# thep = df[df['ICB5']=='Sản xuất, chế biến thép L5']
def add_ratios(x):
    x['jv_income'] = x['jv_inc1'] + x['jv_inc2']
    x['cip'] = x['cip1']
    x['tax_rate'] = 1 - x['net_income'] / (1+x['pretax_inc'])
    x['op'] = x['gp'] - np.abs(x['admin_exp']) - np.abs(x['sel_exp'])
    x['op1'] = x['op'] + x['jv_income']
    x['EBT'] = x['op'] - np.abs(x['int_exp'])
    x['fin_income'] = x['fin_rev'] - (np.abs(x['fin_exp']) - np.abs(x['int_exp']))
    x['other_income'] = x['other_inc']
    # x['bs_gross_fa'] = x['gross_tfa'] + x['gross_ifa'] + x['gross_lease'] + x['gross_ip']
    x['bs_fa'] = x['net_tfa'] + x['net_ifa'] + x['net_ip'] + x['net_lease']
    x['bs_cash'] = (x['cce'] + x['net_st_inv'])
    x['bs_capex'] = -(x.sort_index(level=[0, 1]).groupby(level=0)["bs_fa"].diff(periods=1) +
                      x.sort_index(level=[0, 1]).groupby(level=0)["cip"].diff(periods=1))
    x['bs_ar'] = x['st_ar']
    x['bs_ap'] = x['st_trade_ap']
    x['bs_cust_pre'] = x['st_prepaid_cust'] + x['lt_prepaid_cust'] + x['st_unrev'] + x['lt_unrev']
    x['debt'] = x['st_debt'] + x['lt_debt']
    x['core_e'] = x['op'] * (1 - x['tax_rate'])
    x['op2'] = x['core_e'] + x['jv_income']
    x['netcash'] = (x['bs_cash'] - x['cl'])
    x['ic'] = x['te'] + x['debt']
    x['other_asset'] = x['ta'] - x['bs_cash'] - x['bs_ar'] - x['net_inven'] - x['bs_fa'] - x['cip']-x['lt_inv']
    x['other_lia'] = x['tl'] - x['debt'] - x['bs_ap'] - x['bs_cust_pre']
    x['other_equity'] = x['te'] - x['re'] - x['cap_e'] - x['bs_treasury']
    x['div'] = x['cf_div'] + x['cf_treasury']
    x['cf_delta_debt'] = x['cf_divay'] + x['cf_debt_paid'] + x['cf_lease_paid']
    x['cf_khac'] = (x['cfo'] + x['cfi'] + x['cff']) - (x['cf_capex'] + x['net_income'] + x['cf_dep']
                                                       + x['cf_e_raise'] + x['cf_delta_debt'] + x['div'])
add_ratios(df)
# thep = thep.loc[idx[:,:,5],['rev','gp','sel_exp','admin_exp','jv_inc1','jv_inc2','cf_dep','int_exp','cf_int_paid','st_debt','lt_debt','st_prepaid_cust','st_unrev','lt_prepaid_cust','lt_unrev','ta','te','tl','net_income','bs_fa','bs_cash','debt']]/10**9
# thep.reset_index().to_excel('congtythep.xlsx')

In [11]:
df.loc[idx['TMS',2015:,5],['rev','gp','sel_exp','admin_exp','jv_inc1','jv_inc2','cf_dep','int_exp','cf_int_paid','st_debt','lt_debt','st_prepaid_cust','st_unrev','lt_prepaid_cust','lt_unrev','ta','te','tl','net_income']]/10**9

rev      gp sel_exp admin_exp jv_inc1 jv_inc2  \
Ticker year quarter                                                       
TMS    2015 5.000     488.804 121.547  -0.931   -28.276  85.117   0.000   
       2016 5.000     615.661 154.097  -0.181   -42.425  98.394   0.000   
       2017 5.000   2,131.809 249.327 -13.240   -59.715  99.389   0.000   
       2018 5.000   2,323.712 233.827 -17.006   -68.910 163.927   0.000   
       2019 5.000   2,340.091 272.830 -15.539   -80.137 104.219   0.000   
       2020 5.000   3,418.341 306.873 -12.390   -98.228 170.835   0.000   
       2021 5.000   6,381.266 546.257 -55.163  -159.649 372.116     NaN   
       2022 5.000   3,648.088 549.942 -29.783  -173.874 353.784   0.000   

                     cf_dep int_exp cf_int_paid st_debt lt_debt  \
Ticker year quarter                                               
TMS    2015 5.000    36.007 -15.462      -8.491  58.070 395.281   
       2016 5.000    44.086 -29.492     -44.417 123.836 673.509   
       2017 5.000    66.128 -58.683     -61.291 263.061 517.329   
       2018 5.000    72.019 -62.054     -53.450 194.943 796.924   
       2019 5.000    86.197 -65.984     -70.605 276.926 603.497   
       2020 5.000    95.536 -53.626     -63.248 310.096 533.607   
       2021 5.000   108.302 -54.953     -56.464 716.762 583.265   
       2022 5.000   124.576 -58.089     -59.888 278.622 545.536   

                    st_prepaid_cust st_unrev lt_prepaid_cust lt_unrev  \
Ticker year quarter                                                     
TMS    2015 5.000             0.005    3.075           0.000    0.000   
       2016 5.000             0.036    3.018           0.000    0.000   
       2017 5.000             1.003    2.821           0.000    0.000   
       2018 5.000             2.644    2.480           0.000    0.000   
       2019 5.000             1.627    1.775           0.000    0.000   
       2020 5.000             1.812    0.665           0.000    0.000   
       2021 5.000             3.763    0.536           0.000    0.000   
       2022 5.000             7.051    1.192           0.000    0.000   

                           ta        te        tl net_income  
Ticker year quarter                                           
TMS    2015 5.000   1,420.069   868.259   551.810    155.372  
       2016 5.000   2,006.279 1,078.388   927.890    172.197  
       2017 5.000   2,737.361 1,421.715 1,315.646    211.414  
       2018 5.000   3,166.212 1,625.789 1,540.424    235.012  
       2019 5.000   3,310.259 1,947.165 1,363.094    225.324  
       2020 5.000   3,919.585 2,346.510 1,573.075    321.629  
       2021 5.000   5,734.084 3,489.916 2,244.168    620.900  
       2022 5.000   5,570.603 4,147.586 1,423.018    682.315

In [28]:
df.loc[idx[['GMD','VSC','TMS','SGP','PHP'],2022:,5],['ta','tl','te','debt']]/10**9

,,,ta,tl,te,debt
Ticker,year,quarter,,,,
GMD,2022,5.000,"13,030.653","5,082.897","7,947.756","2,029.579"
VSC,2022,5.000,"4,366.810","1,165.610","3,201.200",908.400
TMS,2022,5.000,"5,570.603","1,423.018","4,147.586",824.158
SGP,2022,5.000,"5,396.366","2,709.199","2,687.167",318.620
PHP,2022,5.000,"6,615.371","1,232.593","5,382.778",577.138


In [ ]:
df[df['ICB5'] == 'Sản xuất, chế biến thép L5'].loc[idx[:,2018:,[1,2,3,4]],['rev','net_income']].groupby(['year','quarter']).apply(lambda x: x.count())
# .loc[idx[['CTD','HBC','FCN','CC1','S99'],2018:,[1,2,3,4]],['net_income']].unstack(0)/10**9
# .groupby(['year','quarter']).apply(lambda x: x['rev'].sum())

# CƠ CẤU CHI PHÍ THEO YẾU TỐ

In [ ]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=42.112.22.56;'
                      'Database=SDv2;'
                      'UID=FiinRate.SDv2.View;'
                      'PWD=FiinR@tings1234')

In [20]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=42.112.22.30,63866;'
                      'Database=FiinRatings;'
                      'UID=FiinRatings.FRA.View;'
                      'PWD=Fiin@1212')

In [21]:
query = """SELECT Distinct
--b.RecordId,
--b.BalanceSheetId,
b.OrganizationId,
a.TaxCode,
a.OrganizationName,
t.Ticker,
P.RealYear as yearreport,
p.RealQuarter as lengthreport,
n.NC160, --chi phi san xuat theo yeu,
n.NC161, --chi phi nguyen lieu vat lieu,
n.NC162, --chi phi nhan cong
n.NC163,--chi phi khau hao tai san co dinh
n.NC164,--chi phi dich vu mua ngoai
n.NC165--chi phi khac bang tien

from [FiinRatings].[dbo].[FGFB_COFS_DR_BalanceSheet] b
join [FiinRatings].[dbo].[FGFB_COIN_DR_ListedInformation] t on b.OrganizationId = t.OrganizationID
left join [FiinRatings].[dbo].[FGFB_COIN_DR_Organization] a on b.OrganizationId = a.OrganizationID
left join [FiinRatings].[dbo].[FGFB_COFS_DR_NoteCompany] n on
b.OrganizationID = n.OrganizationID and 
b.PeriodID = n.PeriodID and 
b.IsPriority = n.IsPriority and 
b.IsHistory = n.IsHistory and
--b.IsPriority = 1 and b.IsHistory = 0 and 
b.ReportTypeId = n.ReportTypeId
INNER JOIN FGFB_SYDI_DR_Period P ON B.PeriodId = P.PeriodId
INNER JOIN FGFB_SYDI_DM_PeriodType Pt ON p.PeriodTypeId = PT.PeriodTypeId
INNER JOIN FGFB_SYDI_DM_FiscalType PF ON p.FiscalTypeId = PF.FiscalTypeId


where 
b.IsPriority = 1 and b.IsHistory = 0
and b.ReportTypeId in (2,3) --2: BCTC hợp nhất kiểm toán, 3: BCTC hợp nhất chưa kiểm toán, 4: BCTC riêng lẻ kiểm toán, 5: BCTC riêng lẻ chưa kiểm toán00
and p.RealQuarter in (1,2,3,4) -- như lengthreport 

--and p.RealYear in (2018,2019,2020,2021,2022) -- đây là year report
and
t.Ticker in (
'SBT'

) 
-- lookup từ mã số thuế theo bảng FGFB__COIN_DR_Organization


"""

In [22]:
df = pd.read_sql_query(query,conn)

In [23]:
note = {'NC160':'tong_chiphi',
        'NC161':'chiphi_nguyenvatlieu',
        'NC162':'chiphi_nhancong',
        'NC163':'chiphi_khauhao',
        'NC164':'chiphi_muangoai',
        'NC165':'chiphi_khac',
       'yearreport':'year',
       'lengthreport':'quarter'}

In [24]:
df = df.rename(columns=note)

In [28]:
df.to_excel('SBT.xlsx')

,OrganizationId,TaxCode,OrganizationName,Ticker,year,quarter,tong_chiphi,chiphi_nguyenvatlieu,chiphi_nhancong,chiphi_khauhao,chiphi_muangoai,chiphi_khac
0,3706,3900244389,Công ty Cổ phần Thành Thành Công - Biên Hòa,SBT,2007,1,NaN,NaN,NaN,NaN,NaN,NaN
1,3706,3900244389,Công ty Cổ phần Thành Thành Công - Biên Hòa,SBT,2007,2,NaN,NaN,NaN,NaN,NaN,NaN
2,3706,3900244389,Công ty Cổ phần Thành Thành Công - Biên Hòa,SBT,2007,3,NaN,NaN,NaN,NaN,NaN,NaN
3,3706,3900244389,Công ty Cổ phần Thành Thành Công - Biên Hòa,SBT,2007,4,NaN,NaN,NaN,NaN,NaN,NaN
4,3706,3900244389,Công ty Cổ phần Thành Thành Công - Biên Hòa,SBT,2008,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
65,3706,3900244389,Công ty Cổ phần Thành Thành Công - Biên Hòa,SBT,2022,3,NaN,NaN,NaN,NaN,NaN,NaN
66,3706,3900244389,Công ty Cổ phần Thành Thành Công - Biên Hòa,SBT,2022,3,0.000,0.000,0.000,0.000,0.000,0.000
67,3706,3900244389,Công ty Cổ phần Thành Thành Công - Biên Hòa,SBT,2022,4,NaN,NaN,NaN,NaN,NaN,NaN
68,3706,3900244389,Công ty Cổ phần Thành Thành Công - Biên Hòa,SBT,2022,4,0.000,0.000,0.000,0.000,0.000,0.000


In [ ]:
df = df.set_index(['Ticker','year','quarter'])

# INDUSTRY RISK SCORING

# CRAWL HOWARD MARKS

## CRAWL CODE

In [20]:
desktop_path = os.path.expanduser("~/Desktop/SuperInvestor/Howard Marks/crawl/")
for i in tnrange(len(x)-1):
    response = requests.get(x.loc[i,0])
    file_name = os.path.basename(x.loc[i,0])
    file_name = re.findall(pattern2, file_name)
    save_path = os.path.join(desktop_path, file_name[0])
    with open(save_path, 'wb') as pdf_file:
        pdf_file.write(response.content)
    

C:\Users\HP PAVILION\AppData\Local\Temp\ipykernel_4824\3483950237.py:2: TqdmDeprecationWarning:

Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`



  0%|          | 0/130 [00:00<?, ?it/s]

NameError: name 'pattern2' is not defined

## HOWARD MARKS - OAKTREE CAPITAL

In [159]:
pdf_folder = os.path.expanduser("~/Desktop/SuperInvestor/Howard Marks/")
output_pdf = 'HowardMarks-Memo-1990_2023.pdf'
pdf_files = []
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_files.append(os.path.join(pdf_folder, filename))
if not pdf_files:
    print("No PDF files found in the directory.")
else:
    pdf_merger = PyPDF2.PdfMerger()
    for pdf_file in pdf_files:
        pdf_merger.append(pdf_file)
    pdf_merger.write(output_pdf)
    pdf_merger.close()

    print(f"Merged PDF saved as '{output_pdf}'")

Merged PDF saved as 'HowardMarks-Memo-1990_2023.pdf'


## Emeth Value Capital

In [160]:
pdf_folder = os.path.expanduser("~/Desktop/SuperInvestor/Emeth Value Capital/")
output_pdf = 'Emeth Value Capital-2018_2023.pdf'
pdf_files = []
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_files.append(os.path.join(pdf_folder, filename))
if not pdf_files:
    print("No PDF files found in the directory.")
else:
    pdf_merger = PyPDF2.PdfMerger()
    for pdf_file in pdf_files:
        pdf_merger.append(pdf_file)
    pdf_merger.write(output_pdf)
    pdf_merger.close()

    print(f"Merged PDF saved as '{output_pdf}'")

Merged PDF saved as 'Emeth Value Capital-2018_2023.pdf'


## St James Investment Company

In [33]:
pdf_folder = os.path.expanduser("~/Desktop/SuperInvestor/St James Investment Company/")
output_pdf = 'St James Investment Company_2014-2023.pdf'
pdf_files = []
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_files.append(os.path.join(pdf_folder, filename))
if not pdf_files:
    print("No PDF files found in the directory.")
else:
    pdf_merger = PyPDF2.PdfMerger()
    for pdf_file in pdf_files:
        pdf_merger.append(pdf_file)
    pdf_merger.write(output_pdf)
    pdf_merger.close()

    print(f"Merged PDF saved as '{output_pdf}'")

Merged PDF saved as 'St James Investment Company_2014-2023.pdf'


In [34]:
pdf_folder = os.path.expanduser("~/Desktop/SuperInvestor/TaoValue/")
output_pdf = 'TaoValue_2017-2021.pdf'
pdf_files = []
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_files.append(os.path.join(pdf_folder, filename))
if not pdf_files:
    print("No PDF files found in the directory.")
else:
    pdf_merger = PyPDF2.PdfMerger()
    for pdf_file in pdf_files:
        pdf_merger.append(pdf_file)
    pdf_merger.write(output_pdf)
    pdf_merger.close()

    print(f"Merged PDF saved as '{output_pdf}'")

Merged PDF saved as 'TaoValue_2017-2021.pdf'


In [35]:
pdf_folder = os.path.expanduser("~/Desktop/SuperInvestor/Righ Tail Capital/")
output_pdf = 'Righ Tail Capital_2022-2023.pdf'
pdf_files = []
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_files.append(os.path.join(pdf_folder, filename))
if not pdf_files:
    print("No PDF files found in the directory.")
else:
    pdf_merger = PyPDF2.PdfMerger()
    for pdf_file in pdf_files:
        pdf_merger.append(pdf_file)
    pdf_merger.write(output_pdf)
    pdf_merger.close()

    print(f"Merged PDF saved as '{output_pdf}'")

Merged PDF saved as 'Righ Tail Capital_2022-2023.pdf'


In [55]:
pdf_folder = os.path.expanduser("~/Desktop/SuperInvestor/Lindsell Train/")
output_pdf = 'Righ Tail Capital_2022-2023.pdf'
pdf_files = []
# for filename in os.listdir(pdf_folder):
#     if filename.endswith('.pdf'):
#         pdf_files.append(os.path.join(pdf_folder, filename))
# if not pdf_files:
#     print("No PDF files found in the directory.")
# else:
#     pdf_merger = PyPDF2.PdfMerger()
#     for pdf_file in pdf_files:
#         pdf_merger.append(pdf_file)
#     pdf_merger.write(output_pdf)
#     pdf_merger.close()

#     print(f"Merged PDF saved as '{output_pdf}'")

## HaydenCapital

In [144]:
import os
dir_path = os.path.expanduser("~/Desktop/SuperInvestor/Hayden Capital/")
file_list = os.listdir(dir_path)
list_file = pd.DataFrame(file_list, columns=['filename'])
list_file = list(list_file['filename'])
x['check'] = x['name'].isin(list_file)

In [145]:
x = x[x['check']==False].reset_index()

In [146]:
def download_pdf(url, save_path):
    headers = {"Accept": "application/pdf",
              "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"}  # Specify that you accept PDF format

    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        with open(save_path, 'wb') as pdf_file:
            pdf_file.write(response.content)
        print(f"PDF downloaded successfully to {save_path}")
    elif response.status_code == 406:
        print("The server cannot provide a response in the format you accept (PDF).")
    else:
        print(f"Failed to download PDF. Status code: {response.status_code}")

In [151]:
desktop_path = os.path.expanduser("~/Desktop/Superinvestor/Hayden Capital/")
for i in tnrange(len(x)):
    save_path = os.path.join(desktop_path, x.loc[i,'name'])
    download_pdf(x.loc[i,'link'], save_path)

C:\Users\HP PAVILION\AppData\Local\Temp\ipykernel_4824\277642095.py:2: TqdmDeprecationWarning:

Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`



  0%|          | 0/22 [00:00<?, ?it/s]

Failed to download PDF. Status code: 404
Failed to download PDF. Status code: 404
Failed to download PDF. Status code: 404
Failed to download PDF. Status code: 404
Failed to download PDF. Status code: 404
Failed to download PDF. Status code: 404
Failed to download PDF. Status code: 404
Failed to download PDF. Status code: 404
Failed to download PDF. Status code: 404
Failed to download PDF. Status code: 404
Failed to download PDF. Status code: 404
Failed to download PDF. Status code: 404
Failed to download PDF. Status code: 404
PDF downloaded successfully to C:\Users\HP PAVILION/Desktop/Superinvestor/Hayden Capital/Hayden-Capital-Quarterly-Letter-2017-Q2.pdf
PDF downloaded successfully to C:\Users\HP PAVILION/Desktop/Superinvestor/Hayden Capital/Hayden-Capital-Quarterly-Letter-2017-Q3.pdf
PDF downloaded successfully to C:\Users\HP PAVILION/Desktop/Superinvestor/Hayden Capital/Hayden-Capital-Quarterly-Letter-2017-Q4.pdf
PDF downloaded successfully to C:\Users\HP PAVILION/Desktop/Superinv

In [152]:
pdf_folder = os.path.expanduser("~/Desktop/SuperInvestor/Hayden Capital/")
output_pdf = 'Hayden_2017-2023.pdf'
pdf_files = []
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_files.append(os.path.join(pdf_folder, filename))
if not pdf_files:
    print("No PDF files found in the directory.")
else:
    pdf_merger = PyPDF2.PdfMerger()
    for pdf_file in pdf_files:
        pdf_merger.append(pdf_file)
    pdf_merger.write(output_pdf)
    pdf_merger.close()

    print(f"Merged PDF saved as '{output_pdf}'")

Merged PDF saved as 'Hayden_2017-2023.pdf'
